In [1]:
import os
path = os.getcwd()
if path == '/content':
    from google.colab import drive
    drive.mount('/content/gdrive')
    BASE_PATH = '/content/gdrive/My Drive/Level-4-Project/'
    !cd gdrive/My\ Drive/Level-4-Project/ && pip install --editable .
    os.chdir('gdrive/My Drive/Level-4-Project/')
    
elif path == 'D:\\Google Drive\\Level-4-Project\\notebooks\\data_processing\\wavenet':
    BASE_PATH = "D:/Google Drive/Level-4-Project/"
    
elif path == "/export/home/2192793m":
    BASE_PATH = "/export/home/2192793m/Level-4-Project/"
    

DATA_PATH = BASE_PATH + 'data/'
RAW_PATH = DATA_PATH + 'raw/raw_converted/'
PROCESSED_PATH = DATA_PATH + 'processed/wavenet/'
    
from src.data import load_data
from src.visualization import multiple_plots, visualize, plot_confusion_matrix
from src.features import make_spectrograms, process_labels, make_directory

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/gdrive
Obtaining file:///content/gdrive/My%20Drive/Level-4-Project
  Running setup.py develop for src


In [0]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import mlab
from matplotlib import colors
from scipy.signal import butter, freqz, lfilter, spectrogram
import time
from sklearn import preprocessing

In [3]:
import numpy as np
import sys
from keras.preprocessing.image import ImageDataGenerator
from keras.optimizers import SGD
from keras.utils import np_utils
import sys
from sklearn.metrics import classification_report, confusion_matrix
import csv
from keras.models import load_model
import pickle

Using TensorFlow backend.


In [0]:
labels = ["walking", "pushing", "sitting", "pulling", "circling", "clapping", "bending"]
processed = {"A":{}, "B":{}, "C":{}, "D":{}, "E":{}, "F":{}}
for name, item in processed.items():
    for label in labels:
        item[label] = []

In [0]:
df_labels = pd.read_csv(RAW_PATH + 'Labels.csv')
df_labels.rename(columns={'dataset ID':'dataset_id'}, inplace=True)
df_labels = process_labels.process_labels(df_labels)

In [0]:
WINDOW_LENGTH = 3 # 3 seconds windows
STEP_SIZE = 1000 # 0.1 second steps

In [0]:
number_of_rows = df_labels.shape[0]
current_row = 1
for row in df_labels.itertuples():
    print(row.user_label, row.aspect_angle, row.label)
    if row.aspect_angle != "0":
        current_row += 1
        continue
        
    print("Processing row", current_row, "of", number_of_rows)
    file_name = RAW_PATH + "Dataset_" + str(row.dataset_id) + ".dat"
    radar_df = pd.read_csv(file_name, header=None)[1]
    
    # Grab RADAR settings from top of file
    center_frequency = float(radar_df.iloc[1])
    sweep_time = float(radar_df.iloc[2])/1000  # convert to seconds
    number_of_time_samples = float(radar_df.iloc[3])
    bandwidth = float(radar_df.iloc[4])
    sampling_frequency = number_of_time_samples/sweep_time
    record_length = 60
    number_of_chirps = record_length/sweep_time
    
    # Put data values into an array
    data = radar_df.iloc[5:].apply(complex).values
    
    # Reshape into chirps over time (means don't need to change step_size calculations just flatten later)
    data_time = np.reshape(data, (int(number_of_chirps),int(number_of_time_samples)))

#     Sanity check to ensure flattening was working correctly
#     plt.plot(abs(data[:384000]))
#     plt.show()

                                         
    window_size = int(WINDOW_LENGTH * 1000)
    iterations = data_time.shape[0] - window_size
    spectrograms = []
    for i in range(0, iterations, STEP_SIZE):
        spectrograms.append(data_time[i:(i + window_size),:].flatten())
        
#     Sanity check part 2        
#     plt.plot(abs(spectrograms[0]))
#     plt.show()

    processed[row.user_label][row.label].append(spectrograms)

    current_row += 1

A 0 walking
Processing row 1 of 123
A 0 walking
Processing row 2 of 123
A 30 walking
A 30 walking
A 60 walking
A 60 walking
A 0 pushing
Processing row 7 of 123
A 0 pushing
Processing row 8 of 123
A 30 pushing
A 30 pushing
A 60 pushing
A 60 pushing
A 0 sitting
Processing row 13 of 123
A 0 sitting
Processing row 14 of 123
A 0 pulling
Processing row 15 of 123
A 0 pulling
Processing row 16 of 123
A 0 circling
Processing row 17 of 123
A 0 circling
Processing row 18 of 123
A 0 clapping
Processing row 19 of 123
A 0 clapping
Processing row 20 of 123
A 0 bending
Processing row 21 of 123
A 0 bending
Processing row 22 of 123
B 0 walking
Processing row 23 of 123
B 0 walking
Processing row 24 of 123
B 0 pushing
Processing row 25 of 123
B 0 pushing
Processing row 26 of 123
B 0 sitting
Processing row 27 of 123
B 0 sitting
Processing row 28 of 123
B 0 circling
Processing row 29 of 123
B 0 circling
Processing row 30 of 123
B 0 clapping
Processing row 31 of 123
B 0 clapping
Processing row 32 of 123
B 0 

In [0]:
# Tried pickle but could not handle the large size. Now try using Numpy save
np.save(PROCESSED_PATH + 'raw_not_reshaped.npy', processed)

In [0]:
# with open(PROCESSED_PATH + "raw_not_reshaped.pkl", "wb") as file:
#     pickle.dump(processed, file)